## **SpeechBrain**

In [ ]:
# Install required packages
!pip install speechbrain torchaudio

In [ ]:
import torchaudio
from speechbrain.inference.speaker import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")



/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [ ]:
signal, fs =torchaudio.load('harvard.wav')
embeddings = classifier.encode_batch(signal)


In [ ]:
signal, fs =torchaudio.load('jackhammer.wav')
embeddings = classifier.encode_batch(signal)

In [ ]:
embeddings.shape

torch.Size([2, 1, 192])

In [ ]:
from speechbrain.inference.speaker import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")


In [ ]:
score, prediction = verification.verify_files('harvard.wav', 'jackhammer.wav') # Different Speakers


In [ ]:
score

tensor([0.4095])

In [ ]:
score, prediction = verification.verify_files("harvard.wav", "harvard.wav") # S

In [ ]:
score

tensor([1.0000])


# **WAV2Vec**

In [ ]:
!pip install torch torchaudio transformers


In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Load the pre-trained Wav2Vec2 model and tokenizer for speech-to-text (ASR)
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Function to process audio and extract embeddings or transcription
def process_audio(file_path):
    # Load audio file with torchaudio
    waveform, sample_rate = torchaudio.load(file_path)

    # Resample the audio to 16 kHz (required by Wav2Vec2)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    # Normalize the audio waveform
    input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", sampling_rate=16000).input_values

    # Get the logits (raw predictions) from the model
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode the logits to text (transcription)
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])

    return transcription, logits

# Example usage with an audio file
audio_file = 'harvard.wav'
transcription, logits = process_audio(audio_file)

print("Transcription:", transcription)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Transcription: THE STALE SMELL OF OLD BEER LINGERS IT TAKES HEAT TO BRING OUT THE ODOR A COLD DIP RESTORES HEALTH AND ZEST A SALT PICKLE TASTES FINE WITH HAM TAKOZ AL PASTORE ARE MY FAVORITE A ZESTFUL FOOD IS THE HOT CROSS BUN


In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained Wav2Vec2 model for feature extraction
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name)

# Function to process audio and extract embeddings
def extract_embeddings(file_path):
    # Load the audio file
    waveform, sample_rate = torchaudio.load(file_path)

    # Resample the audio to 16 kHz if necessary
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    # Normalize and preprocess the audio
    input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", sampling_rate=16000).input_values

    # Extract embeddings using Wav2Vec2Model
    with torch.no_grad():
        embeddings = model(input_values).last_hidden_state

    # Average the embeddings across the sequence length (dimension 1) to get a single vector
    mean_embeddings = embeddings.mean(dim=1).squeeze().numpy()

    return mean_embeddings

# Extract embeddings for the two audio files
embeddings_harvard = extract_embeddings('harvard.wav')
embeddings_jackhammer = extract_embeddings('jackhammer.wav')

# Ensure that embeddings are 2D before computing cosine similarity
embeddings_harvard = embeddings_harvard.reshape(1, -1)
embeddings_jackhammer = embeddings_jackhammer.reshape(1, -1)

# Compute the cosine similarity between the two embedding vectors
similarity = cosine_similarity(embeddings_harvard, embeddings_jackhammer)

print(f"Cosine Similarity between the two audio files: {similarity[0][0]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Cosine Similarity between the two audio files: 0.8959


## **SQLLITE3**



In [ ]:
!pip install torch torchaudio transformers sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [ ]:
import torch
import torchaudio
import sqlite3
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files

# Load the pre-trained Wav2Vec2 model for feature extraction
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name)

# Initialize SQLite database connection
conn = sqlite3.connect('audio_embeddings.db')
cursor = conn.cursor()

# Create a table for storing embeddings if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS audio_embeddings (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        file_name TEXT,
        embedding BLOB
    )
''')
conn.commit()

# Function to process audio and extract embeddings with a fixed size
def extract_embeddings(file_path, fixed_embedding_size=768):
    # Load the audio file
    waveform, sample_rate = torchaudio.load(file_path)

    # Resample the audio to 16 kHz if necessary
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    # Normalize and preprocess the audio
    input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", sampling_rate=16000).input_values

    # Extract embeddings using Wav2Vec2Model
    with torch.no_grad():
        embeddings = model(input_values).last_hidden_state

    # Average the embeddings across the sequence length (dimension 1) to get a single vector
    mean_embeddings = embeddings.mean(dim=1).squeeze().numpy()

    # Ensure the embedding has a fixed size by truncating or padding with zeros
    if mean_embeddings.shape[0] > fixed_embedding_size:
        mean_embeddings = mean_embeddings[:fixed_embedding_size]
    elif mean_embeddings.shape[0] < fixed_embedding_size:
        mean_embeddings = np.pad(mean_embeddings, (0, fixed_embedding_size - mean_embeddings.shape[0]), 'constant')

    return mean_embeddings

# Function to store embeddings in the SQLite database
def store_embedding(file_name, embedding):
    # Convert the numpy array to binary data to store in the database
    embedding_blob = embedding.tobytes()
    cursor.execute('INSERT INTO audio_embeddings (file_name, embedding) VALUES (?, ?)', (file_name, embedding_blob))
    conn.commit()

# Function to find the most similar audio from the database for a given embedding
def find_most_similar(embedding):
    cursor.execute('SELECT file_name, embedding FROM audio_embeddings')
    rows = cursor.fetchall()

    max_similarity = -1
    most_similar_file = None

    for row in rows:
        stored_file_name = row[0]
        stored_embedding = np.frombuffer(row[1], dtype=np.float32)

        # Ensure both embeddings have the same size
        if stored_embedding.shape[0] != embedding.shape[0]:
            continue  # Skip comparison if the sizes don't match

        # Compute the cosine similarity between the new and stored embeddings
        similarity = cosine_similarity([embedding], [stored_embedding])[0][0]

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_file = stored_file_name

    return most_similar_file, max_similarity

# Upload audio files using Colab's file upload utility
uploaded_files = files.upload()

# Process and store the uploaded audio files
for file_name in uploaded_files.keys():
    embedding = extract_embeddings(file_name)
    store_embedding(file_name, embedding)
    print(f"Stored embedding for {file_name} in the database.")

# Test similarity with a new audio file upload
print("Please upload a new audio file for similarity testing:")
new_file = files.upload()
new_file_name = list(new_file.keys())[0]
new_embedding = extract_embeddings(new_file_name)

most_similar_file, similarity_score = find_most_similar(new_embedding)
print(f"The most similar audio file to {new_file_name} is {most_similar_file} with a similarity score of {similarity_score:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Saving a1.wav to a1 (1).wav
Saving a2.wav to a2 (1).wav
Saving a3.wav to a3 (1).wav
Saving a4.wav to a4 (1).wav
Saving a5.wav to a5 (1).wav
Stored embedding for a1 (1).wav in the database.
Stored embedding for a2 (1).wav in the database.
Stored embedding for a3 (1).wav in the database.
Stored embedding for a4 (1).wav in the database.
Stored embedding for a5 (1).wav in the database.
Please upload a new audio file for similarity testing:


Saving 2.wav to 2 (1).wav
The most similar audio file to 2 (1).wav is a1.wav with a similarity score of 0.9931


s1---a3 voice but different content

In [ ]:
# Import the required file upload module
from google.colab import files

# Upload a new audio file for similarity testing
print("Please upload a new audio file for similarity testing:")
new_file = files.upload()

# Extract the filename of the uploaded file
new_file_name = list(new_file.keys())[0]

# Call the function to extract embeddings and find the most similar audio file
new_embedding = extract_embeddings(new_file_name)
most_similar_file, similarity_score = find_most_similar(new_embedding)

# Display the results of the similarity test
print(f"The most similar audio file to {new_file_name} is {most_similar_file} with a similarity score of {similarity_score:.4f}")


Please upload a new audio file for similarity testing:


Saving s1.wav to s1.wav
The most similar audio file to s1.wav is a3.wav with a similarity score of 0.9740


s2--new voice ---s1 content

In [ ]:
# Import the required file upload module
from google.colab import files

# Upload a new audio file for similarity testing
print("Please upload a new audio file for similarity testing:")
new_file = files.upload()

# Extract the filename of the uploaded file
new_file_name = list(new_file.keys())[0]

# Call the function to extract embeddings and find the most similar audio file
new_embedding = extract_embeddings(new_file_name)
most_similar_file, similarity_score = find_most_similar(new_embedding)

# Display the results of the similarity test
print(f"The most similar audio file to {new_file_name} is {most_similar_file} with a similarity score of {similarity_score:.4f}")


Please upload a new audio file for similarity testing:


Saving s2.wav to s2.wav
The most similar audio file to s2.wav is a1.wav with a similarity score of 0.9767


R1---

In [ ]:
# Import the required file upload module
from google.colab import files

# Upload a new audio file for similarity testing
print("Please upload a new audio file for similarity testing:")
new_file = files.upload()

# Extract the filename of the uploaded file
new_file_name = list(new_file.keys())[0]

# Call the function to extract embeddings and find the most similar audio file
new_embedding = extract_embeddings(new_file_name)
most_similar_file, similarity_score = find_most_similar(new_embedding)

# Display the results of the similarity test
print(f"The most similar audio file to {new_file_name} is {most_similar_file} with a similarity score of {similarity_score:.4f}")


Please upload a new audio file for similarity testing:


Saving r1.wav to r1.wav
The most similar audio file to r1.wav is a5.wav with a similarity score of 0.9745


2==same a1

In [ ]:
# Import the required file upload module
from google.colab import files

# Upload a new audio file for similarity testing
print("Please upload a new audio file for similarity testing:")
new_file = files.upload()

# Extract the filename of the uploaded file
new_file_name = list(new_file.keys())[0]

# Call the function to extract embeddings and find the most similar audio file
new_embedding = extract_embeddings(new_file_name)
most_similar_file, similarity_score = find_most_similar(new_embedding)

# Display the results of the similarity test
print(f"The most similar audio file to {new_file_name} is {most_similar_file} with a similarity score of {similarity_score:.4f}")


Please upload a new audio file for similarity testing:


Saving 2.wav to 2 (2).wav
The most similar audio file to 2 (2).wav is a1.wav with a similarity score of 0.9931


In [ ]:
# Import the required file upload module
from google.colab import files

# Upload a new audio file for similarity testing
print("Please upload a new audio file for similarity testing:")
new_file = files.upload()

# Extract the filename of the uploaded file
new_file_name = list(new_file.keys())[0]

# Call the function to extract embeddings and find the most similar audio file
new_embedding = extract_embeddings(new_file_name)
most_similar_file, similarity_score = find_most_similar(new_embedding)

# Display the results of the similarity test
print(f"The most similar audio file to {new_file_name} is {most_similar_file} with a similarity score of {similarity_score:.4f}")


Please upload a new audio file for similarity testing:


Saving r2.wav to r2.wav
The most similar audio file to r2.wav is a2.wav with a similarity score of 0.9726
